In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from datetime import timedelta
from functools import lru_cache

In [3]:
os.getcwd()

'c:\\Users\\caarch25\\Desktop\\RiskAnalysisProject\\MF728-RiskAnalysisProject\\Main Coding File'

In [4]:
data = pd.read_csv("../Data/CouponBonds/CouponBondData/YieldData/A Yield.csv", index_col = 0,header = list(range(8)))

data.sort_index(inplace = True)
data.index = pd.to_datetime(data.index)

data = data.loc["2020-01-01":"2020-12-31"]

In [6]:
class CIR_base:
    """
    CIR model class that takes in a series of yields.
    """

    def __init__(self, rates: pd.Series):
        """
        Initialize the CIR model with a series of yields.
        
        Parameters
        ----------
        rates : pd.DataFrame
            A DataFrame containing the yield data.
        """
        self.rates = rates

    def calibrate(self):
        """Calibrate CIR model parameters using OLS regression."""
        self.rates, rates = self.rates/100, self.rates/100

        rt = rates[1:]

        #setting up the parameters for OLS regression
        delta_rt = rates.diff().dropna()
        sqrt_rt = np.sqrt(rt)
        sqrt_delta_t = np.sqrt(pd.Series(rates.index.to_series().diff().dropna().dt.days.values, index=delta_rt.index) / 360 * 12)  # Ensure matching lengths

        #setting up the OLS regression
        y = delta_rt/(sqrt_rt * sqrt_delta_t)
        Y = np.array(y).reshape(-1, 1)

        x1 = sqrt_delta_t/sqrt_rt
        x2 = sqrt_rt*sqrt_delta_t

        X = np.array([x1, x2]).T

        # Fit the linear regression model
        model = LinearRegression(fit_intercept=False)
        model.fit(X, Y)

        # Extract the coefficients
        a, b = model.coef_[0]

        residuals = Y - model.predict(X)

        # Calculate the parameters
        kappa = -b
        theta = a/kappa
        
        self.model = model
        self.theta = theta
        self.kappa = kappa
        self.sigma = residuals.std()

    def simulate(self, N:int = 1, starting_rate:float = None) ->np.ndarray:
        """
        Simulate the CIR process using calibrated parameters.
        
        Parameters
        ----------
        N : int
            Number of simulations to run.
        starting_rate : float
            Starting interest rate for the simulation. If None, uses the first rate in the series.

        Returns
        -------
        np.ndarray
            Simulated interest rates. Each row corresponds to a simulation, and each column corresponds to a time step.
        """
        # Calibrate the CIR process
        if not hasattr(self, 'kappa'):
            self.calibrate()

        if starting_rate is None:
            starting_rate = self.rates.iloc[0]

        # Getting CIR parameters
        theta = self.theta
        kappa = self.kappa
        sigma = self.sigma
        delta_t = 1/360

        rates = np.zeros((N, len(self.rates)))

        for i in range(N):
            # Simulate the process
            # Use the CIR model parameters to generate future rates


            Z = np.random.normal(0, 1, size=(len(self.rates)))

            rates[i, :] = np.zeros(len(self.rates))
            rates[i, 0] = starting_rate

            for t in range(1, len(self.rates)):
                rates[i, t] = rates[i,t-1] + kappa * (theta - rates[i,t-1]) * delta_t + sigma * np.sqrt(delta_t) * np.sqrt(rates[i,t-1]) * Z[t-1]

        return rates
    
class CIR(CIR_base):
    """
    CIR model class that takes in a series of yields.
    """

    def __init__(self, rates: pd.DataFrame):
        """
        Initialize the CIR model with a series of yields.
        
        Parameters
        ----------
        rates : pd.DataFrame
            A DataFrame containing the yield data.
        """

        self.rates = rates/100

        
        self.dataframe = rates.apply(lambda x : CIR_base(x), axis=0)

    def calibrate(self):
        self.dataframe.apply(lambda x : x.calibrate())
    
    def simulate(self, starting_rates:pd.Series|None = None) ->np.ndarray:
        """
        Simulate the CIR process using calibrated parameters.
        
        Parameters
        ----------
        N : int
            Number of simulations to run.
        starting_rate : float
            Starting interest rate for the simulation. If None, uses the first rate in the series.

        Returns
        -------
        np.ndarray
            Simulated interest rates. Each row corresponds to a simulation, and each column corresponds to a time step.
        """
        # Calibrate the CIR process
        if not hasattr(self, 'dataframe'):
            self.calibrate()

        if starting_rates is None:
            simulated_rates = self.dataframe.apply(lambda x : pd.Series(x.simulate()[0])).T
            starting_date = self.rates.index[0]
        else:
            if len(starting_rates) != len(self.dataframe):
                raise ValueError("Starting rates must have the same length as the dataframe.")
            
            starting_date = starting_rates.name
            starting_rates = starting_rates/100

            index = [starting_date + pd.Timedelta(days=i) for i in range(len(self.rates))]

            simulated_rates = pd.DataFrame(columns=self.dataframe.index, index = index)
            
            for i, column in enumerate(starting_rates.index):

                starting_rate = starting_rates[column]
                model = self.dataframe[column]

                # simulated_rates[column] = model.simulate(starting_rate=starting_rate)
                simulated_array = model.simulate(starting_rate=starting_rate)[0]

                simulated_rates[column] = simulated_array
        
        return simulated_rates
    

model = CIR(data)
model.calibrate()

simulations = model.simulate(data.iloc[-1,:])

simulations

,341099CH0 Corp,92976GAJ0 Corp,983024AN0 Corp,74456QAR7 Corp,207597DX0 Corp,771367BZ1 Corp,695114CG1 Corp,641423BU1 Corp,039483AX0 Corp,797440BJ2 Corp,...,92976GAJ0 Corp_20,695114CG1 Corp_21,695114CG1 Corp_22,207597DX0 Corp_23,87612EAR7 Corp,89417EAD1 Corp,695114CG1 Corp_26,207597DX0 Corp_27,141781AW4 Corp,171232AQ4 Corp
Company,Duke Energy Florida LLC,Wells Fargo Bank NA,Wyeth LLC,Public Service Electric and Gas Co,Connecticut Light and Power Co/The,Rochester Gas and Electric Corp,PacifiCorp,Nevada Power Co,Archer-Daniels-Midland Co,San Diego Gas & Electric Co,...,Wells Fargo Bank NA,PacifiCorp,PacifiCorp,Connecticut Light and Power Co/The,Target Corp,Travelers Cos Inc/The,PacifiCorp,Connecticut Light and Power Co/The,Cargill Inc,Chubb Corp/The
Ticker,DUK,WFC,PFE,PEG,ES,AGR,BRKHEC,BRKHEC,ADM,SRE,...,WFC,BRKHEC,BRKHEC,ES,TGT,TRV,BRKHEC,ES,CARGIL,CB
Rating,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
Issue Date,2007-09-18 00:00:00,2007-12-07 00:00:00,2007-03-27 00:00:00,2007-05-14 00:00:00,2007-03-27 00:00:00,2007-07-17 00:00:00,2007-10-03 00:00:00,2007-06-28 00:00:00,2007-12-11 00:00:00,2007-09-20 00:00:00,...,2007-12-07 00:00:00,2007-10-03 00:00:00,2007-10-03 00:00:00,2007-03-27 00:00:00,2007-10-05 00:00:00,2007-05-29 00:00:00,2007-10-03 00:00:00,2007-03-27 00:00:00,2007-09-11 00:00:00,2007-05-11 00:00:00
Maturity Date,2037-09-15 00:00:00,2038-01-15 00:00:00,2037-04-01 00:00:00,2037-05-01 00:00:00,2037-03-01 00:00:00,2032-07-15 00:00:00,2037-10-15 00:00:00,2037-07-01 00:00:00,2038-01-15 00:00:00,2037-09-15 00:00:00,...,2038-01-15 00:00:00,2037-10-15 00:00:00,2037-10-15 00:00:00,2037-03-01 00:00:00,2037-10-15 00:00:00,2037-06-15 00:00:00,2037-10-15 00:00:00,2037-03-01 00:00:00,2037-09-15 00:00:00,2037-05-11 00:00:00
Coupon,6.35,6.6,5.95,5.8,5.75,6.47,6.25,6.75,6.45,6.125,...,6.6,6.25,6.25,5.75,6.5,6.25,6.25,5.75,6.625,6
Tenor (days),10955,10997,10963,10945,10932,9130,10970,10961,10993,10953,...,10997,10970,10970,10932,10968,10975,10970,10932,10962,10958
2020-12-31,0.215530,0.215530,0.215530,0.215530,0.215530,0.215530,0.215530,0.215530,0.215530,0.215530,...,0.215530,0.215530,0.215530,0.215530,0.215530,0.215530,0.215530,0.215530,0.215530,0.215530
2021-01-01,0.210616,0.212148,0.220863,0.221318,0.212622,0.215207,0.215164,0.211228,0.216466,0.216407,...,0.218285,0.214456,0.212490,0.215884,0.213606,0.216260,0.220292,0.215382,0.217059,0.217281
2021-01-02,0.212193,0.214589,0.218045,0.215309,0.213408,0.218948,0.215684,0.210303,0.212560,0.212897,...,0.220622,0.212619,0.210535,0.218188,0.210988,0.218640,0.222097,0.215941,0.209921,0.220432


In [ ]:
data.loc["2020-01-01":"2020-12-31"]

,341099CH0 Corp,92976GAJ0 Corp,983024AN0 Corp,74456QAR7 Corp,207597DX0 Corp,771367BZ1 Corp,695114CG1 Corp,641423BU1 Corp,039483AX0 Corp,797440BJ2 Corp,...,92976GAJ0 Corp_20,695114CG1 Corp_21,695114CG1 Corp_22,207597DX0 Corp_23,87612EAR7 Corp,89417EAD1 Corp,695114CG1 Corp_26,207597DX0 Corp_27,141781AW4 Corp,171232AQ4 Corp
Company,Duke Energy Florida LLC,Wells Fargo Bank NA,Wyeth LLC,Public Service Electric and Gas Co,Connecticut Light and Power Co/The,Rochester Gas and Electric Corp,PacifiCorp,Nevada Power Co,Archer-Daniels-Midland Co,San Diego Gas & Electric Co,...,Wells Fargo Bank NA,PacifiCorp,PacifiCorp,Connecticut Light and Power Co/The,Target Corp,Travelers Cos Inc/The,PacifiCorp,Connecticut Light and Power Co/The,Cargill Inc,Chubb Corp/The
Ticker,DUK,WFC,PFE,PEG,ES,AGR,BRKHEC,BRKHEC,ADM,SRE,...,WFC,BRKHEC,BRKHEC,ES,TGT,TRV,BRKHEC,ES,CARGIL,CB
Rating,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
Issue Date,2007-09-18 00:00:00,2007-12-07 00:00:00,2007-03-27 00:00:00,2007-05-14 00:00:00,2007-03-27 00:00:00,2007-07-17 00:00:00,2007-10-03 00:00:00,2007-06-28 00:00:00,2007-12-11 00:00:00,2007-09-20 00:00:00,...,2007-12-07 00:00:00,2007-10-03 00:00:00,2007-10-03 00:00:00,2007-03-27 00:00:00,2007-10-05 00:00:00,2007-05-29 00:00:00,2007-10-03 00:00:00,2007-03-27 00:00:00,2007-09-11 00:00:00,2007-05-11 00:00:00
Maturity Date,2037-09-15 00:00:00,2038-01-15 00:00:00,2037-04-01 00:00:00,2037-05-01 00:00:00,2037-03-01 00:00:00,2032-07-15 00:00:00,2037-10-15 00:00:00,2037-07-01 00:00:00,2038-01-15 00:00:00,2037-09-15 00:00:00,...,2038-01-15 00:00:00,2037-10-15 00:00:00,2037-10-15 00:00:00,2037-03-01 00:00:00,2037-10-15 00:00:00,2037-06-15 00:00:00,2037-10-15 00:00:00,2037-03-01 00:00:00,2037-09-15 00:00:00,2037-05-11 00:00:00
Coupon,6.35,6.6,5.95,5.8,5.75,6.47,6.25,6.75,6.45,6.125,...,6.6,6.25,6.25,5.75,6.5,6.25,6.25,5.75,6.625,6
Tenor (days),10955,10997,10963,10945,10932,9130,10970,10961,10993,10953,...,10997,10970,10970,10932,10968,10975,10970,10932,10962,10958
2020-01-02,10.273,10.273,10.273,10.273,10.273,10.273,10.273,10.273,10.273,10.273,...,10.273,10.273,10.273,10.273,10.273,10.273,10.273,10.273,10.273,10.273
2020-01-03,9.872,9.872,9.872,9.872,9.872,9.872,9.872,9.872,9.872,9.872,...,9.872,9.872,9.872,9.872,9.872,9.872,9.872,9.872,9.872,9.872
2020-01-06,10.070,10.070,10.070,10.070,10.070,10.070,10.070,10.070,10.070,10.070,...,10.070,10.070,10.070,10.070,10.070,10.070,10.070,10.070,10.070,10.070
